In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model


In [2]:
df_user = pd.read_csv('../../dataset/user_rating.csv')
place_num = len(df_user.Place_Id.unique())
n_users = len(df_user.User_Id.unique())



In [3]:
df_destination = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';', header=0)
df_destination['price'].fillna(0, inplace=True)
df_destination['category'].fillna('unknown', inplace=True)

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_user, test_size=0.2, random_state=42)

In [5]:
def create_model():
    # creating destination embedding path
    destination_input = Input(shape=[1])
    destination_embedding = Embedding(place_num+1, 5)(destination_input)
    destination_flat = Flatten()(destination_embedding)

    # creating user embedding path
    user_input = Input(shape=[1])
    user_embedding = Embedding(n_users+1, 5)(user_input)
    user_flat = Flatten()(user_embedding)

    # concatenate features
    conc = Concatenate()([destination_flat, user_flat])

    # add fully-connected-layers
    x = Dense(128, activation='relu')(conc)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)

    # Create model and compile it
    model = Model([user_input, destination_input], output)
    model.compile('adam', 'mean_squared_error')
    history = model.fit([train.User_Id, train.Place_Id], train.Place_Ratings, epochs=20)
    return history, model

Predict

In [6]:
bali_dataset = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';')
bali_dataset
# recommended_places = bali_dataset.loc[recommended_tourism_ids, 'place']
# recommended_places

,index,place,url,is_accessibility,rating,n_reviews,address,price,idx_category,category,description,lat,long,Coordinate
0,0,Pura Puseh Desa Adat Kutri,https://www.google.com/maps/place/Pura+Puseh+D...,0,3.0,1,"F863+W53, Buruan, Kec. Blahbatuh, Kabupaten Gi...",15000,0,Sejarah,Pura Puseh is one of the oldest temples in Bal...,-8.514987,115.298791,"-8.514986706754593, 115.29879103124946"
1,1,Pura Dalem Begawan,https://www.google.com/maps/place/Pura+Dalem+B...,0,3.0,2,"8682+87P, Dauh Puri Kauh, Kec. Denpasar Bar., ...",15000,0,Sejarah,"Located near the Petanu River, Pura Dalem Bega...",-8.682596,115.200610,"-8.682595632877103, 115.20061015733499"
2,2,D'tukad Desa Budaya Kertalangu,https://www.google.com/maps/place/D'tukad+Desa...,0,3.6,9,"9744+WJQ, Kesiman Kertalangu, Kec. Denpasar Ti...",20000,0,Sejarah,D'tukad Desa Budaya Kertalangu is a cultural t...,-8.639347,115.257474,"-8.639347199121982, 115.25747408289917"
3,3,Desa Adat Cangkup,https://www.google.com/maps/place/Desa+Adat+Ca...,0,3.7,3,"Pupuan Sawah, Kec. Selemadeg, Kabupaten Tabana...",15000,0,Sejarah,Desa Adat Cangkup is a traditional village tha...,-8.439189,115.051934,"-8.43918883821229, 115.05193408611053"
4,4,Monumen Perjuangan Desa Tegaljadi,https://www.google.com/maps/place/Monumen+Perj...,0,4.0,1,"G585+J3, Tegaljadi, Kec. Marga, Kabupaten Taba...",15000,0,Sejarah,This monument was erected to commemorate the s...,-8.483242,115.157706,"-8.4832418767373, 115.15770565767143"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,190,Banjar Hot Spring,https://www.google.com/maps/place/Banjar+Hot+S...,0,4.3,2422,"Jalan Banjar, Banjar, Kec. Banjar, Kabupaten B...",20000,1,Wisata Alam,Hot springs complex surrounded by tropical gar...,-8.210255,114.967123,"-8.210254599982365, 114.96712273733684"
191,191,Bali Pulina,https://www.google.com/maps/place/Bali+Pulina/...,0,4.5,3960,"Jl. Raya Pujung Kaja No.Br, Sebatu, Kec. Tegal...",100000,1,Wisata Alam,"Sprawling venue with lush, tropical vegetation...",-8.422674,115.278990,"-8.422673949154303, 115.27899012017349"
192,192,Goa Lawah Temple,https://www.google.com/maps/place/Goa+Lawah+Te...,0,4.7,4479,"CFX9+9GG, Jl. Raya Goa Lawah, Pesinggahan, Kec...",20000,0,Sejarah,"This small, 11th-century Balinese Hindu temple...",-8.551260,115.468823,"-8.551260117939812, 115.46882253550854"
193,193,Pantai Batu Bolong,https://www.google.com/maps/place/Pantai+Batu+...,0,4.4,7248,"Jl. Pantai Batu Bolong, Canggu, Kec. Kuta Utar...",15000,2,Pantai,This well-known sandy beach near shops & resta...,-8.658103,115.130329,"-8.658103348069142, 115.13032921850571"


In [7]:
# history, model = create_model()

In [87]:
def create_final_data(list_of_idx):
    list_of_dest = []
    
    for row in list_of_idx:
        dest_dict = {}
        get_data = df_destination.iloc[row]
        dest_dict['idx_place'] = get_data['index']
        dest_dict['place'] = get_data['place']
        dest_dict['url'] = get_data['url']
        dest_dict['address'] = get_data['address']
        dest_dict['is_accessibility'] = get_data['is_accessibility']
        dest_dict['rating'] = get_data['rating']
        dest_dict['n_reviews'] = get_data['n_reviews']
        dest_dict['price'] = get_data['price']
        dest_dict['category'] = get_data['category']
        dest_dict['description'] = get_data['description']
        list_of_dest.append(dest_dict)
    
    return list_of_dest

In [91]:
df_destination.iloc[5]

index                                                               5
place               Monumen Perjuangan Masyarakat Bali (Perang Pup...
url                 https://www.google.com/maps/place/Monumen+Perj...
is_accessibility                                                    0
rating                                                            4.0
n_reviews                                                           2
address             86V9+94R, Dauh Puri Kangin, Kec. Denpasar Bar....
price                                                           15000
idx_category                                                        0
category                                                      Sejarah
description         This monument commemorates the Puputan War, a ...
lat                                                         -8.655522
long                                                       115.217651
Coordinate                      -8.65552218189213, 115.21765127469615
Name: 5, dtype: obje

In [89]:
# model.save('../model.h5')
model = tf.keras.models.load_model('../model.h5')


In [90]:
def might_like(idx_user, category):
    new_df = df_destination[df_destination['idx_category'].isin(category)].copy()

    # create prediction
    tourism_data = np.array(list(set(df_user.Place_Id)))
    user = np.array([idx_user]*len(tourism_data))
    predictions = model.predict([user, tourism_data])
    predictions = np.array([a[0] for a in predictions])
    
    recommended_tourism_idx = (-predictions).argsort()
    # print("this is recommended_tourism_idx",recommended_tourism_idx)
    recommended_tourism_idx = [idx for idx in recommended_tourism_idx if idx in new_df.index][:10]
    print("this is recommended_tourism_idx after",recommended_tourism_idx)
    #convert index to data place
    data = create_final_data(recommended_tourism_idx)
    return data

In [86]:
idx_user = 5
category = [2,3]
get_recom = might_like(idx_user, category)
get_recom

7/7 [==============================] - 0s 8ms/step
this is recommended_tourism_idx after [138, 18, 87, 9, 63, 71, 75, 47, 35, 118]


[{'idx_place': 138,
  'place': 'Pantai Batu Mejan',
  'url': 'https://www.google.com/maps/place/Mejan+Stone+Beach/@-8.6546834,115.1044994,14z/data=!3m1!4b1!4m6!3m5!1s0x2dd2387669f265eb:0xf287030294190338!8m2!3d-8.6547265!4d115.1250991!16s%2Fg%2F11g6n_1k24?entry=ttu',
  'address': 'GWFJ+RP4, Jalan Pantai Mejan Jl. Surabrata, Lalanglinggah, Kec. Selemadeg Bar., Kabupaten Tabanan, Bali 82162',
  'is_accessibility': 0,
  'rating': 4.8,
  'n_reviews': 16,
  'price': 15000,
  'category': 'Pantai',
  'description': 'Batu Mejan Beach, also known as Echo Beach, is one of the popular beaches in Canggu, Bali. This beach is famous for its big waves and is suitable for surfing. In addition, this beach also has beautiful scenery and a calm atmosphere, making it an ideal place to relax and enjoy the sunset.'},
 {'idx_place': 18,
  'place': 'Dermaga Cinta',
  'url': 'https://www.google.com/maps/place/Dermaga+Cinta/@-8.1597145,115.0230595,17z/data=!3m1!4b1!4m6!3m5!1s0x2dd19b3fcba612df:0x7bcde688a0f74a2